In [7]:
import pandas as pd
import plotly.graph_objects as go

# Rewards

In [8]:
def plot_rewards(chunk_size, *paths):
    fig = go.Figure()
    fig.update_xaxes(title_text='Iterations')
    fig.update_yaxes(title_text='Mean Reward')
    fig.update_layout(title_text="Reward")
    COLORS = ["#FF9AA2", "#C7CEEA", "#B5EAD7", "#FFB7B2", "#E2F0CB", "#FFDAC1"]
    for i, path in enumerate(paths):
        data =pd.read_csv(f'{path[0]}/reward.csv', header=None)
        data.columns = ['iteration', 'value']
        chunk_size = chunk_size
        batch_size = 32
        data =data.groupby(lambda idx: (idx+1) // chunk_size).agg({'iteration': ['max'], 'value': 'sum'})
        data.columns = data.columns.droplevel(1)
        data['value'] /= (chunk_size * batch_size)
        fig.add_trace(go.Scatter(x=data['iteration'][:-1], y=data['value'][:-1],
                            mode='lines+markers', name=path[1],
                            marker=dict(color=COLORS[i], opacity=0.3,),
                            line=dict(width=1,)
                      ))
    fig.show()

In [9]:
plot_rewards(5000, ('logs', 'voc_size_89'), ('logs/first_training', 'ref'), ('logs/third_training', 'ref2'))

# Gradient Flow

In [12]:
from IPython.display import display
def show_gradient_flow(path):
    data = pd.read_csv(f'{path}/grad_flow.csv', header=None)
    data[2] = data[2].str.replace('tensor\(', '').str.replace(", device='cuda:0'\)", '').str.replace("\)", '').astype('float64')
    data[3] = data[3].str.replace('tensor\(', '').str.replace(", device='cuda:0'\)", '').str.replace("\)", '').astype('float64')
    data.columns = ['iteration', 'layer', 'mean_grad', 'max_grad']    
    display(data[['layer', 'mean_grad', 'max_grad']].groupby('layer').agg({'mean_grad':['max', 'mean', 'median'], 'max_grad':['max', 'mean', 'median']}))
    show_gradient_flow_plot(data, 'encoder')
    show_gradient_flow_plot(data, 'decoder')
    show_gradient_flow_plot(data, 'generator')


def show_gradient_flow_plot(data, layer_type):
    fig = go.Figure()
    fig.update_xaxes(title_text='Iterations')
    fig.update_yaxes(title_text='Mean Gradient', range=[0, 1e-6])
    fig.update_layout(title_text=f"{layer_type.title()} Gradients")
    for layer in data['layer'].unique():
        if layer.startswith(layer_type):
            fig.add_trace(go.Scatter(x=data[data['layer'] == layer]['iteration'], y=data[data['layer'] == layer]['mean_grad'],
                                mode='lines+markers', name=layer, marker=dict(
                        opacity=0.5,
                    )))
    fig.show()

In [13]:
show_gradient_flow('logs')

mean_grad  \
                                                             max   
layer                                                              
decoder.attn.linear_in.weight                       5.855600e-06   
decoder.attn.linear_out.weight                      3.858000e-06   
decoder.rnn.layers.0.weight_hh                      1.021300e-06   
decoder.rnn.layers.0.weight_ih                      1.859200e-06   
decoder_embeddings.make_embedding.emb_luts.0.we...  1.075400e-06   
encoder.rnn.weight_hh_l0                            9.810100e-07   
encoder.rnn.weight_hh_l0_reverse                    8.955800e-07   
encoder.rnn.weight_ih_l0                            8.113300e-06   
encoder.rnn.weight_ih_l0_reverse                    7.808200e-06   
encoder_embeddings.make_embedding.emb_luts.0.we...  1.803300e-06   
generator.advantages.0.weight_mu                    2.416300e-06   
generator.advantages.0.weight_sigma                 1.641900e-06   
generator.advantages.2.weight_mu                    2.629400e-07   
generator.advantages.2.weight_sigma                 1.847000e-07   
generator.value.0.weight_mu                         2.755400e-06   
generator.value.0.weight_sigma                      1.966300e-06   
generator.value.2.weight_mu                         1.064100e-05   
generator.value.2.weight_sigma                      6.950400e-06   

                                                                  \
                                                            mean   
layer                                                              
decoder.attn.linear_in.weight                       1.398288e-07   
decoder.attn.linear_out.weight                      3.343872e-07   
decoder.rnn.layers.0.weight_hh                      6.604403e-08   
decoder.rnn.layers.0.weight_ih                      1.090483e-07   
decoder_embeddings.make_embedding.emb_luts.0.we...  6.421521e-08   
encoder.rnn.weight_hh_l0                            6.330874e-08   
encoder.rnn.weight_hh_l0_reverse                    6.482341e-08   
encoder.rnn.weight_ih_l0                            3.591116e-07   
encoder.rnn.weight_ih_l0_reverse                    3.681862e-07   
encoder_embeddings.make_embedding.emb_luts.0.we...  9.767941e-08   
generator.advantages.0.weight_mu                    2.369176e-07   
generator.advantages.0.weight_sigma                 1.602947e-07   
generator.advantages.2.weight_mu                    1.990822e-08   
generator.advantages.2.weight_sigma                 1.349647e-08   
generator.value.0.weight_mu                         1.097099e-07   
generator.value.0.weight_sigma                      7.435957e-08   
generator.value.2.weight_mu                         6.358044e-07   
generator.value.2.weight_sigma                      4.300117e-07   

                                                                  max_grad  \
                                                          median       max   
layer                                                                        
decoder.attn.linear_in.weight                       6.257850e-08  0.000085   
decoder.attn.linear_out.weight                      2.534400e-07  0.000200   
decoder.rnn.layers.0.weight_hh                      4.597100e-08  0.000048   
decoder.rnn.layers.0.weight_ih                      7.244000e-08  0.000200   
decoder_embeddings.make_embedding.emb_luts.0.we...  4.011750e-08  0.000093   
encoder.rnn.weight_hh_l0                            4.735000e-08  0.000022   
encoder.rnn.weight_hh_l0_reverse                    4.861800e-08  0.000028   
encoder.rnn.weight_ih_l0                            2.486150e-07  0.000200   
encoder.rnn.weight_ih_l0_reverse                    2.450700e-07  0.000200   
encoder_embeddings.make_embedding.emb_luts.0.we...  6.661500e-08  0.000100   
generator.advantages.0.weight_mu                    2.049000e-07  0.000064   
generator.advantages.0.weight_sigma                 1.386850e-07  0.000100   
generator.advantages.2.weig

# Loss

In [11]:
def show_loss(path):
    data = pd.read_csv(f'{path}/loss.csv', header=None)
    data.columns = ['iteration', 'value']
    data = data.groupby(lambda idx: (idx+1) // 3).agg({'iteration': ['max'], 'value': 'sum'})
    data.columns = data.columns.droplevel(1)
    fig = go.Figure()
    fig.update_xaxes(title_text='Iterations')
    fig.update_yaxes(title_text='Loss')
    fig.update_layout(title_text='Loss')
    fig.add_trace(go.Scatter(x=data['iteration'][0:-1], y=data['value'][0:-1],
                        mode='lines+markers', name='markers', marker=dict(
                color='MediumPurple',
                opacity=0.5,
            )))
    fig.show()

In [12]:
show_loss('logs')

# Sigma Magnitudes

In [13]:
def show_sigma(path):
    data =pd.read_csv(f'{path}/sig_param_mag.csv', header=None)
    data.columns = ['iteration', 'value']
    fig = go.Figure()
    fig.update_xaxes(title_text='Iterations')
    fig.update_yaxes(title_text='Mean Absolute Sigma')
    fig.update_layout(title_text='Sigma')
    fig.add_trace(go.Scatter(x=data['iteration'], y=data['value'],
                        mode='lines+markers', name='markers', marker=dict(
                color='MediumPurple',
                opacity=0.5,
            )))
    fig.show()

In [14]:
show_sigma('logs')

# Action Selection

In [15]:
def show_action_selection(path):
    data = pd.read_csv(f'{path}/sample.csv', header=None)
    data.columns = ['iteration', 'value']
    data['value'] = data['value'].str.split('(').str.get(0)
    data['value'] = data['value'].str.strip().str.split(' ')
    vals = pd.get_dummies(data['value'].apply(pd.Series).stack()).sum(level=0)
    vals.drop(columns=['<s>', '</s>'], inplace=True)
    data = data.join(vals)
    data = data.groupby(by='iteration').sum().reset_index()
    data = data.groupby(lambda idx: (idx+1) // 20).sum()
    data = data[:-1]
    probabilities = data[data.columns[1:]].div(data[data.columns[1:]].sum(axis=1), axis='index')
    probabilities = probabilities.reindex(probabilities.median().sort_values().index, axis=1)
    probabilities_ = probabilities[probabilities.columns[-14:]] 
    probabilities_['Other'] = probabilities[probabilities.columns[:-14]].sum(axis=1)
    data = data[['iteration']].join(probabilities_)
    data['iteration'] = data['iteration'] / 20
    fig = go.Figure()
    fig.update_xaxes(title_text='Iterations')
    fig.update_yaxes(title_text='Mean Reward')
    fig.update_layout(title_text="Reward")
    for column in data.columns[1:]:
        fig.add_trace(go.Scatter(x=data['iteration'], y=data[column],
                                mode='lines+markers', name=column,
                                marker=dict(opacity=0.3,),
                                line=dict(width=1,),
                                stackgroup='one'
                          ))
    fig.show()

In [16]:
show_action_selection("logs")

c:\users\fabian\.virtualenvs\rl-adventure-xhdax2fk\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

